# Interacting with Campaigns <a class="anchor" id="top"></a>

In this notebook, you will deploy and interact with campaigns in Amazon Personalize.

1. [Introduction](#intro)
1. [Create campaigns](#create)
1. [Interact with campaigns](#interact)
1. [Batch recommendations](#batch)
1. [Wrap up](#wrapup)

## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

At this point, you should have several solutions and at least one solution version for each. Once a solution version is created, it is possible to get recommendations from them, and to get a feel for their overall behavior.

This notebook starts off by deploying each of the solution versions from the previous notebook into individual campaigns. Once they are active, there are resources for querying the recommendations, and helper functions to digest the output into something more human-readable. 

As you with your customer on Amazon Personalize, you can modify the helper functions to fit the structure of their data input files to keep the additional rendering working.

To get started, once again, we need to import libraries, load values from previous notebooks, and load the SDK.

In [12]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random

import boto3
import pandas as pd

In [13]:
%store -r

In [14]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Interact with campaigns <a class="anchor" id="interact"></a>
[Back to top](#top)

Now that all campaigns are deployed and active, we can start to get recommendations via an API call. Each of the campaigns is based on a different recipe, which behave in slightly different ways because they serve different use cases. We will cover each campaign in a different order than used in previous notebooks, in order to deal with the possible complexities in ascending order (i.e. simplest first).

First, let's create a supporting function to help make sense of the results returned by a Personalize campaign. Personalize returns only an `item_id`. This is great for keeping data compact, but it means you need to query a database or lookup table to get a human-readable result for the notebooks. We will create a helper function to return a human-readable result from the LastFM dataset.

Start by loading in the dataset which we can use for our lookup table.

In [15]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(dataset_dir + '/movies.csv', sep=',', usecols=[0,1], encoding='latin-1', dtype={'movieId': "object", 'title': "str"},index_col=0)

# Render some sample data
items_df.head(5)

,title
movieId,
1,Toy Story (1995)
2,Jumanji (1995)
3,Grumpier Old Men (1995)
4,Waiting to Exhale (1995)
5,Father of the Bride Part II (1995)


In [16]:
items_df.dtypes

title    object
dtype: object

By defining the ID column as the index column it is trivial to return an artist by just querying the ID. Movie #589 should be Terminator 2: Judgment Day.

In [17]:
movie_id_example = 589
title = items_df.loc[movie_id_example]['title']
print(title)

Terminator 2: Judgment Day (1991)


That isn't terrible, but it would get messy to repeat this everywhere in our code, so the function below will clean that up.

In [18]:
def get_movie_by_id(movie_id, movie_df=items_df):
    """
    This takes in an artist_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return movie_df.loc[int(movie_id)]['title']
    except:
        return "Error obtaining title"

Now let's test a few simple values to check our error catching.

In [19]:
# A known good id (The Princess Bride)
print(get_movie_by_id(movie_id="1197"))
# A bad type of value
print(get_movie_by_id(movie_id="987.9393939"))
# Really bad values
print(get_movie_by_id(movie_id="Steve"))

Princess Bride, The (1987)
Error obtaining title
Error obtaining title


Great! Now we have a way of rendering results. 

### SIMS

SIMS requires just an item as input, and it will return items which users interact with in similar ways to their interaction with the input item. In this particular case the item is a movie. 

The cells below will handle getting recommendations from SIMS and rendering the results. Let's see what the recommendations are for the first item we looked at earlier in this notebook (Terminator 2: Judgment Day).

In [20]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = sims_campaign_arn,
    itemId = str(589),
)

In [21]:
item_list = get_recommendations_response['itemList']
for item in item_list:
    print(get_movie_by_id(movie_id=item['itemId']))

Jurassic Park (1993)
Braveheart (1995)
Terminator, The (1984)
Fugitive, The (1993)
Speed (1994)
Crimson Tide (1995)
GoldenEye (1995)
True Lies (1994)
Batman (1989)
Clear and Present Danger (1994)
Mask, The (1994)
Die Hard: With a Vengeance (1995)
In the Line of Fire (1993)
Lion King, The (1994)
Ghost (1990)
Forrest Gump (1994)
Apollo 13 (1995)
Cliffhanger (1993)
Star Trek: Generations (1994)
Firm, The (1993)
Die Hard (1988)
Mission: Impossible (1996)
Seven (a.k.a. Se7en) (1995)
Indiana Jones and the Last Crusade (1989)
Mrs. Doubtfire (1993)


Congrats, this is your first list of recommendations! This list is fine, but it would be better to see the recommendations for our sample collection of artists render in a nice dataframe. Again, let's create a helper function to achieve this.

In [22]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df(recommendations_df, movie_ID):
    # Get the movie name
    movie_name = get_movie_by_id(movie_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = sims_campaign_arn,
        itemId = str(movie_ID),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

Now, let's test the helper function with several different movies. Let's sample some data from our dataset to test our SIMS campaign. Grab 5 random movies from our dataframe.

Note: We are going to show similar titles, so you may want to re-run the sample until you recognize some of the movies listed

In [23]:
samples = items_df.sample(5)
samples

,title
movieId,
2174,Beetlejuice (1988)
5110,Super Troopers (2001)
5440,She Wore a Yellow Ribbon (1949)
3447,"Good Earth, The (1937)"
91104,"Twilight Saga: Breaking Dawn - Part 1, The (2011)"


In [24]:
sims_recommendations_df = pd.DataFrame()
movies = samples.index.tolist()

for movie in movies:
    sims_recommendations_df = get_new_recommendations_df(sims_recommendations_df, movie)

sims_recommendations_df

,Beetlejuice (1988),Super Troopers (2001),She Wore a Yellow Ribbon (1949),"Good Earth, The (1937)","Twilight Saga: Breaking Dawn - Part 1, The (2011)"
0,Big (1988),Idiocracy (2006),"Shawshank Redemption, The (1994)",Lilies of the Field (1963),Bride Wars (2009)
1,Austin Powers: International Man of Mystery (1...,What We Do in the Shadows (2014),Forrest Gump (1994),Elmer Gantry (1960),"Twilight Saga: Eclipse, The (2010)"
2,"Last Unicorn, The (1982)","Babadook, The (2014)",Pulp Fiction (1994),"Longest Yard, The (1974)",Something Borrowed (2011)
3,Lost in Austen (2008),Hachiko: A Dog's Story (a.k.a. Hachi: A Dog's ...,"Silence of the Lambs, The (1991)","Searchers, The (1956)","Family Stone, The (2005)"
4,"Breakfast Club, The (1985)",How High (2001),Braveheart (1995),Harley Davidson and the Marlboro Man (1991),Confessions of a Shopaholic (2009)
5,"Nutty Professor, The (1963)",Grandma's Boy (2006),"Matrix, The (1999)",Spartacus (1960),"Best Exotic Marigold Hotel, The (2011)"
6,There's Something About Mary (1998),Deuce Bigalow: European Gigolo (2005),Schindler's List (1993),High Noon (1952),Where the Heart Is (2000)
7,Defending Your Life (1991),"Hot Chick, The (2002)",Star Wars: Episode IV - A New Hope (1977),Little Big Man (1970),"Paris, I Love You (Paris, je t'aime) (2006)"
8,Chitty Chitty Bang Bang (1968),Grind (2003),Jurassic Park (1993),"Hustler, The (1961)",Friends with Benefits (2011)
9,"Blues Brothers, The (1980)",Inglorious Bastards (Quel maledetto treno blin...,Terminator 2: Judgment Day (1991),Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),Friday Night Lights (2004)


You may notice that a lot of the items look the same, hopefully not all of them do (this is more likely with a smaller # of interactions, which will be more common with the movielens small dataset). This shows that the evaluation metrics should not be the only thing you rely on when evaluating your solution version. So when this happens, what can you do to improve the results?

This is a good time to think about the hyperparameters of the Personalize recipes. The SIMS recipe has a `popularity_discount_factor` hyperparameter (see [documentation](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-sims.html)). Leveraging this hyperparameter allows you to control the nuance you see in the results. This parameter and its behavior will be unique to every dataset you encounter, and depends on the goals of the business. You can iterate on the value of this hyperparameter until you are satisfied with the results, or you can start by leveraging Personalize's hyperparameter optimization (HPO) feature. For more information on hyperparameters and HPO tuning, see the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/customizing-solution-config-hpo.html).

### User Personalization

HRNN is one of the more advanced algorithms provided by Amazon Personalize. It supports personalization of the items for a specific user based on their past behavior and can intake real time events in order to alter recommendations for a user without retraining. 

Since HRNN relies on having a sampling of users, let's load the data we need for that and select 3 random users. Since Movielens does not include user data, we will select 3 random numbers from the range of user id's in the dataset.

In [25]:
if not USE_FULL_MOVIELENS:
    users = random.sample(range(1, 600), 3)
else:
    users = random.sample(range(1, 162000), 3)
users

[251, 112, 214]

Now we render the recommendations for our 3 random users from above. After that, we will explore real-time interactions before moving on to Personalized Ranking.

Again, we create a helper function to render the results in a nice dataframe.

#### API call results

In [26]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_users(recommendations_df, user_id):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [27]:
recommendations_df_users = pd.DataFrame()
#users = users_df.sample(3).index.tolist()

for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

,251,112,214
0,Reservoir Dogs (1992),The Imitation Game (2014),Phenomenon (1996)
1,Bowling for Columbine (2002),Inglourious Basterds (2009),Tin Cup (1996)
2,Forrest Gump (1994),Coraline (2009),Up Close and Personal (1996)
3,Jurassic Park (1993),Kill Bill: Vol. 1 (2003),Primal Fear (1996)
4,Dr. Strangelove or: How I Learned to Stop Worr...,District 9 (2009),Boogie Nights (1997)
5,American History X (1998),Children of Men (2006),Eraser (1996)
6,Casino Royale (2006),"Pursuit of Happyness, The (2006)",James and the Giant Peach (1996)
7,"Silence of the Lambs, The (1991)",WALLÂ·E (2008),William Shakespeare's Romeo + Juliet (1996)
8,Little Miss Sunshine (2006),"40-Year-Old Virgin, The (2005)","Time to Kill, A (1996)"
9,"Inconvenient Truth, An (2006)","Hangover, The (2009)",Multiplicity (1996)


Here we clearly see that the recommendations for each user are different. If you were to need a cache for these results, you could start by running the API calls through all your users and store the results, or you could use a batch export, which will be covered later in this notebook.

### Static and Dynamic Filters

Lets interact with the static filters we created in the previous notebook, and utilize dynamic filters in realtime.

A few common use cases for dynamic filters in Video On Demand are:

Categorical filters based on Item Metadata (that arent range based) - Often your item metadata will have information about the title such as Genre, Keyword, Year, Director, Actor etc. Filtering on these can provide recommendations within that data, such as action movies, Steven Spielberg movies, Movies from 1995 etc.

Events - you may want to filter out certain events and provide results based on those events, such as moving a title from a "suggestions to watch" recommendation to a "watch again" recommendations.

Now lets apply item filters to see recommendations for one of these users within each decade of our static filters.


In [28]:
def get_new_recommendations_df_by_static_filter(recommendations_df, user_id, filter_arn):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
        filterArn = filter_arn
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    filter_name = filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [29]:
def get_new_recommendations_df_by_dynamicfilter(recommendations_df, user_id, genre_filter_arn, filter_values):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
        filterArn = genre_filter_arn,
        filterValues = { "GENRE": "\"" + filter_values + "\""}
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    filter_name = filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_values])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

You can see the recommendations for movies within a given decade. Within a VOD application you could create Shelves (also known as rails or carosels) easily by using these filters. Depending on the information you have about your items, You could also filter on additional information such as keyword, year/decade etc.

In [36]:
recommendations_df_decade_shelves = pd.DataFrame()
for filter_arn in decade_filter_arns:
    recommendations_df_decade_shelves = get_new_recommendations_df_by_static_filter(recommendations_df_decade_shelves, user, filter_arn)

recommendations_df_decade_shelves.head(10)

,1950s,1960s,1970s,1980s,1990s,2000s,2010s
0,"Streetcar Named Desire, A (1951)",2001: A Space Odyssey (1968),Willy Wonka & the Chocolate Factory (1971),E.T. the Extra-Terrestrial (1982),Phenomenon (1996),Erin Brockovich (2000),Inception (2010)
1,"King and I, The (1956)",Bonnie and Clyde (1967),Star Wars: Episode IV - A New Hope (1977),Terms of Endearment (1983),Tin Cup (1996),"Lord of the Rings: The Return of the King, The...","Twilight Saga: Breaking Dawn - Part 2, The (2012)"
2,12 Angry Men (1957),"Graduate, The (1967)","Godfather, The (1972)",Big (1988),Up Close and Personal (1996),Cast Away (2000),Rock of Ages (2012)
3,"African Queen, The (1951)","Sound of Music, The (1965)",One Flew Over the Cuckoo's Nest (1975),Manon of the Spring (Manon des sources) (1986),Primal Fear (1996),Dancer in the Dark (2000),Harry Potter and the Deathly Hallows: Part 2 (...
4,Vertigo (1958),West Side Story (1961),"Candidate, The (1972)",Dead Poets Society (1989),Boogie Nights (1997),Finding Forrester (2000),"MisÃ©rables, Les (2012)"
5,"20,000 Leagues Under the Sea (1954)",Easy Rider (1969),Star Trek: The Motion Picture (1979),Do the Right Thing (1989),Eraser (1996),"Lord of the Rings: The Fellowship of the Ring,...",Holy Motors (2012)
6,Cat on a Hot Tin Roof (1958),Cool Hand Luke (1967),Harold and Maude (1971),"Honey, I Shrunk the Kids (1989)",James and the Giant Peach (1996),Big Fish (2003),Thor (2011)
7,"Bridge on the River Kwai, The (1957)",Mary Poppins (1964),Kramer vs. Kramer (1979),Star Wars: Episode VI - Return of the Jedi (1983),William Shakespeare's Romeo + Juliet (1996),No Man's Land (2001),Cloud Atlas (2012)
8,Seven Samurai (Shichinin no samurai) (1954),Midnight Cowboy (1969),"Tin Drum, The (Blechtrommel, Die) (1979)",Top Gun (1986),"Time to Kill, A (1996)",You Got Served (2004),12 Years a Slave (2013)
9,All About Eve (1950),Breakfast at Tiffany's (1961),Rocky (1976),Field of Dreams (1989),Multiplicity (1996),Under the Tuscan Sun (2003),Temple Grandin (2010)


In [37]:
# Create a dataframe for the items by reading in the correct source CSV
items_meta_df = pd.read_csv(data_dir + '/item-meta.csv', sep=',', index_col=0)

# Render some sample data
items_meta_df.head(10)

,GENRE,YEAR,CREATION_TIMESTAMP
ITEM_ID,,,
1,Adventure|Animation|Children|Comedy|Fantasy,1995,0
2,Adventure|Children|Fantasy,1995,0
3,Comedy|Romance,1995,0
4,Comedy|Drama|Romance,1995,0
5,Comedy,1995,0
6,Action|Crime|Thriller,1995,0
7,Comedy|Romance,1995,0
8,Adventure|Children,1995,0
9,Action,1995,0


Now what we want to do is determine the genres to filter on, for that we need a list of all genres. First we will get all the unique values of the column GENRE, then split strings on | if they exist, everyone will then get added to a long list which will be converted to a set for efficiency. That set will then be made into a list so that it can be iterated, and we can then use the get recommendatioins API.

In [38]:
unique_genre_field_values = items_meta_df['GENRE'].unique()

genre_val_list = []

def process_for_bar_char(val, val_list):
    if '|' in val:
        values = val.split('|')
        for item in values:
            val_list.append(item)
    elif '(' in val:
        pass
    else:
        val_list.append(val)
    return val_list
    

for val in unique_genre_field_values:
    genre_val_list = process_for_bar_char(val, genre_val_list)

genres_to_filter = list(set(genre_val_list))

In [39]:
genres_to_filter

['Drama',
 'Film-Noir',
 'Mystery',
 'Horror',
 'Musical',
 'Crime',
 'Animation',
 'Western',
 'Romance',
 'Fantasy',
 'Action',
 'Thriller',
 'Children',
 'Sci-Fi',
 'Comedy',
 'War',
 'Documentary',
 'IMAX',
 'Adventure']

In [40]:
# Iterate through Genres
recommendations_df_genre_shelves = pd.DataFrame()
for genre in genres_to_filter:
    recommendations_df_genre_shelves = get_new_recommendations_df_by_dynamicfilter(recommendations_df_genre_shelves, user, genre_filter_arn , genre)
    
recommendations_df_genre_shelves

,Drama,Film-Noir,Mystery,Horror,Musical,Crime,Animation,Western,Romance,Fantasy,Action,Thriller,Children,Sci-Fi,Comedy,War,Documentary,IMAX,Adventure
0,Phenomenon (1996),2 Days in the Valley (1996),Primal Fear (1996),"Craft, The (1996)",James and the Giant Peach (1996),Primal Fear (1996),James and the Giant Peach (1996),Lone Star (1996),Phenomenon (1996),James and the Giant Peach (1996),Eraser (1996),Primal Fear (1996),James and the Giant Peach (1996),Star Trek: First Contact (1996),Tin Cup (1996),"English Patient, The (1996)",Everest (1998),Everest (1998),James and the Giant Peach (1996)
1,Tin Cup (1996),Lost Highway (1997),Lone Star (1996),"Frighteners, The (1996)","Hunchback of Notre Dame, The (1996)",Fargo (1996),"Hunchback of Notre Dame, The (1996)",Maverick (1994),Tin Cup (1996),Dragonheart (1996),White Squall (1996),Eraser (1996),"Hunchback of Notre Dame, The (1996)",Mystery Science Theater 3000: The Movie (1996),Multiplicity (1996),Courage Under Fire (1996),Looking for Richard (1996),Apollo 13 (1995),White Squall (1996)
2,Up Close and Personal (1996),Bitter Moon (1992),Conspiracy Theory (1997),Mary Reilly (1996),Evita (1996),Courage Under Fire (1996),Space Jam (1996),Legends of the Fall (1994),Up Close and Personal (1996),Michael (1996),Star Trek: First Contact (1996),"Time to Kill, A (1996)",Matilda (1996),"Nutty Professor, The (1996)",Emma (1996),Saving Private Ryan (1998),From the Earth to the Moon (1998),More (1998),Star Trek: First Contact (1996)
3,Primal Fear (1996),Hoodlum (1997),Midnight in the Garden of Good and Evil (1997),Scream (1996),Willy Wonka & the Chocolate Factory (1971),Striptease (1996),All Dogs Go to Heaven 2 (1996),Almost Heroes (1998),William Shakespeare's Romeo + Juliet (1996),Matilda (1996),Courage Under Fire (1996),"Juror, The (1996)",Willy Wonka & the Chocolate Factory (1971),Star Trek: Insurrection (1998),Mighty Aphrodite (1995),Richard III (1995),Paradise Lost: The Child Murders at Robin Hood...,T-Rex: Back to the Cretaceous (1998),Dragonheart (1996)
4,Boogie Nights (1997),"This World, Then the Fireworks (1997)","Red Violin, The (Violon rouge, Le) (1998)",Tales from the Crypt Presents: Bordello of Blo...,All Dogs Go to Heaven 2 (1996),Mulholland Falls (1996),Hercules (1997),Unforgiven (1992),Emma (1996),"Nutty Professor, The (1996)",Dragonheart (1996),Star Trek: First Contact (1996),Space Jam (1996),Space Jam (1996),Private Parts (1997),"MisÃ©rables, Les (1995)",T-Rex: Back to the Cretaceous (1998),Alaska: Spirit of the Wild (1997),"Phantom, The (1996)"
5,Eraser (1996),L.A. Confidential (1997),Amistad (1997),Thinner (1996),Selena (1997),Dead Man Walking (1995),Princess Mononoke (Mononoke-hime) (1997),Dead Man (1995),Jerry Maguire (1996),Pleasantville (1998),G.I. Jane (1997),Fargo (1996),101 Dalmatians (1996),"City of Lost Children, The (CitÃ© des enfants ...",Beautiful Girls (1996),Seven Years in Tibet (1997),Alaska: Spirit of the Wild (1997),"Lion King, The (1994)",Executive Decision (1996)
6,William Shakespeare's Romeo + Juliet (1996),Dark City (1998),"City of Lost Children, The (CitÃ© des enfants ...","Faculty, The (1998)",Everyone Says I Love You (1996),Midnight in the Garden of Good and Evil (1997),Aladdin and the King of Thieves (1996),Geronimo: An American Legend (1993),Mighty Aphrodite (1995),Willy Wonka & the Chocolate Factory (1971),"Phantom, The (1996)",Mulholland Falls (1996),Flipper (1996),Mars Attacks! (1996),Spy Hard (1996),"Thin Red Line, The (1998)",Year of the Horse (1997),Beauty and the Beast (1991),Space Jam (1996)
7,"Time to Kill, A (1996)",Devil in a Blue Dress (1995),"Game, The (1997)",Wes Craven's New Nightmare (Nightmare on Elm S...,Hercules (1997),Trainspotting (1996),Anastasia (1997),Dances with Wolves (1990),Beautiful Girls (1996),Space Jam (1996),Executive Decision (1996),Eye for an Eye (1996),All Dogs Go to Heaven 2 (1996),Contact (1997),Chasing Amy (1997),Welcome to Sarajevo (1997),Hands on a Hard Body (1996),"Matrix Revolutions, The (2003)","City of Lost Chil

The next topic is real-time events. Personalize has the ability to listen to events from your application in order to update the recommendations shown to the user. This is especially useful in media workloads, like video-on-demand, where a customer's intent may differ based on if they are watching with their children or on their own.

Additionally the events that are recorded via this system are stored until a delete call from you is issued, and they are used as historical data alongside the other interaction data you provided when you train your next models.

#### Real time events

Start by creating an event tracker that is attached to the campaign.

In [41]:
response = personalize.create_event_tracker(
    name='MovieTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']
event_tracker_arn = response['eventTrackerArn']

arn:aws:personalize:us-east-1:832194813872:event-tracker/dff308a5
4add244b-085f-4119-858c-69a20a2005da


We will create some code that simulates a user interacting with a particular item. After running this code, you will get recommendations that differ from the results above.

We start by creating some methods for the simulation of real time events.

In [42]:
session_dict = {}

def send_movie_click(USER_ID, ITEM_ID, EVENT_TYPE):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[str(USER_ID)]
    except:
        session_dict[str(USER_ID)] = str(uuid.uuid1())
        session_ID = session_dict[str(USER_ID)]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= str(USER_ID),
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': str(EVENT_TYPE),
        'properties': event_json
        }]
    )

def get_new_recommendations_df_users_real_time(recommendations_df, user_id, item_id, event_type):
    # Get the artist name (header of column)
    movie_name = get_movie_by_id(item_id)
    # Interact with different movies
    print('sending event ' + event_type + ' for ' + get_movie_by_id(item_id))
    send_movie_click(USER_ID=user_id, ITEM_ID=item_id, EVENT_TYPE=event_type)
    # Get the recommendations (note you should have a base recommendation DF created before)
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        artist = get_movie_by_id(item['itemId'])
        recommendation_list.append(artist)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    #recommendations_df = recommendations_df.join(new_rec_DF)
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

At this point, we haven't generated any real-time events yet; we have only set up the code. To compare the recommendations before and after the real-time events, let's pick one user and generate the original recommendations for them.

In [43]:
# First pick a user
user_id = user

# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    artist = get_movie_by_id(item['itemId'])
    recommendation_list.append(artist)
user_recommendations_df = pd.DataFrame(recommendation_list, columns = [user_id])
user_recommendations_df

,214
0,Phenomenon (1996)
1,Tin Cup (1996)
2,Up Close and Personal (1996)
3,Primal Fear (1996)
4,Boogie Nights (1997)
5,Eraser (1996)
6,James and the Giant Peach (1996)
7,William Shakespeare's Romeo + Juliet (1996)
8,"Time to Kill, A (1996)"
9,Multiplicity (1996)


Ok, so now we have a list of recommendations for this user before we have applied any real-time events. Now let's pick 3 random artists which we will simulate our user interacting with, and then see how this changes the recommendations.

In [44]:
# Next generate 3 random movies
movies = items_df.sample(3).index.tolist()

In [45]:
# Note this will take about 15 seconds to complete due to the sleeps
for movie in movies:
    user_recommendations_df = get_new_recommendations_df_users_real_time(user_recommendations_df, user_id, movie,'click')
    time.sleep(5)

sending event click for Mack, The (1973)
sending event click for Bogus (1996)
sending event click for Babes in Toyland (1934)


Now we can look at how the click events changed the recommendations.

In [46]:
user_recommendations_df

,214,"Mack, The (1973)",Bogus (1996),Babes in Toyland (1934)
0,Phenomenon (1996),Phenomenon (1996),Primal Fear (1996),Phenomenon (1996)
1,Tin Cup (1996),Tin Cup (1996),Phenomenon (1996),James and the Giant Peach (1996)
2,Up Close and Personal (1996),Up Close and Personal (1996),Up Close and Personal (1996),Primal Fear (1996)
3,Primal Fear (1996),Primal Fear (1996),James and the Giant Peach (1996),Up Close and Personal (1996)
4,Boogie Nights (1997),Boogie Nights (1997),Courage Under Fire (1996),Striptease (1996)
5,Eraser (1996),Eraser (1996),Striptease (1996),Tin Cup (1996)
6,James and the Giant Peach (1996),James and the Giant Peach (1996),Multiplicity (1996),Courage Under Fire (1996)
7,William Shakespeare's Romeo + Juliet (1996),William Shakespeare's Romeo + Juliet (1996),2 Days in the Valley (1996),Multiplicity (1996)
8,"Time to Kill, A (1996)","Time to Kill, A (1996)",Tin Cup (1996),Eraser (1996)
9,Multiplicity (1996),Multiplicity (1996),Dead Man Walking (1995),"Time to Kill, A (1996)"


In the cell above, the first column after the index is the user's default recommendations from User Personalization, and each column after that has a header of the artist that they interacted with via a real time event, and the recommendations after this event occurred. 

The behavior may not shift very much; this is due to the relatively limited nature of this dataset and effect of a few random clicks. If you wanted to better understand this, try simulating clicking more movies, and you should see a more pronounced impact.

Now lets look at the event filters, which allow you to filter items based on the interaction data. For this dataset, it could be click or watch based on the data we imported, but could be based on whatever interaction schema you design (click, rate, like, watch, purchase etc.) For VOD shelves you could move a title from "Top picks for you" to a "Watch again", the watch again recommendations will be based on the users current interactions, but only recommend titles that have already been watched.


In [48]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_recommendations_df_by_static_filter(recommendations_df_events, user, filter_arn)
    
recommendations_df_events

InvalidInputException: An error occurred (InvalidInputException) when calling the GetRecommendations operation: The given filter must be in the ACTIVE state: arn:aws:personalize:us-east-1:832194813872:filter/watched

now lets send a watch event in for 4 unwatched recommendations, which would simulate watching 4 movies. In a VOD application, you may choose to send in an event after they have watched a significant amount (over 75%) of a piece of content. Sending at 100% complete could miss people that stop short of the credits.

In [41]:
 # Get the recommendations
top_unwatched_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = userpersonalization_campaign_arn,
    userId = str(user_id),
    filterArn = filter_arn,
    numResults=4)
item_list = top_unwatched_recommendations_response['itemList']
for item in item_list:
    print('sending event watch for ' + get_movie_by_id(item['itemId']))
    send_movie_click(USER_ID=user_id, ITEM_ID=item['itemId'], EVENT_TYPE='watch')
    time.sleep(10)

sending event watch for Waking Life (2001)
sending event watch for Spirited Away (Sen to Chihiro no kamikakushi) (2001)
sending event watch for Billy Elliot (2000)
sending event watch for Men of Honor (2000)


Now we can look at the event filters to see the updated watched and unwatched recommendations 

In [42]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_recommendations_df_by_static_filter(recommendations_df_events, user, filter_arn)
    
recommendations_df_events

,watched,unwatched
0,Billy Elliot (2000),"Truman Show, The (1998)"
1,Pulp Fiction (1994),"Wolf of Wall Street, The (2013)"
2,Almost Famous (2000),Pitch Perfect (2012)
3,Spirited Away (Sen to Chihiro no kamikakushi) ...,"Help, The (2011)"
4,"Shawshank Redemption, The (1994)","Hunt, The (Jagten) (2012)"
5,Waking Life (2001),Star Trek Into Darkness (2013)
6,Fight Club (1999),Black Swan (2010)
7,Lost in Translation (2003),50/50 (2011)
8,One Flew Over the Cuckoo's Nest (1975),Warrior (2011)
9,Forrest Gump (1994),Snowpiercer (2013)


### Personalized Ranking

The core use case for personalized ranking is to take a collection of items and to render them in priority or probable order of interest for a user. For a VOD application you want dynamically render a personalized shelf/rail/carousel based on some information (director, location, superhero franchise, movie time period etc). This may not be information that you have in your metadata, so a item metadata filter will not work, howeverr you may have this information within you system to generate the item list. 

To demonstrate this, we will use the same user from before and a random collection of items.

In [49]:
rerank_user = user
rerank_items = items_df.sample(25).index.tolist()

Now build a nice dataframe that shows the input data.

In [50]:
rerank_list = []
for item in rerank_items:
    movie = get_movie_by_id(item)
    rerank_list.append(movie)
rerank_df = pd.DataFrame(rerank_list, columns = ['Un-Ranked'])
rerank_df

,Un-Ranked
0,Babylon 5: The Legend of the Rangers: To Live ...
1,"First Wives Club, The (1996)"
2,Boniface's Holiday (1965)
3,Exotica (1994)
4,Horrors of Spider Island (Ein Toter Hing im Ne...
5,The Emperor's Club (2002)
6,"Ladykillers, The (2004)"
7,"Fall of the House of Usher, The (House of Ushe..."
8,"Queen of Versailles, The (2012)"
9,Dragon Ball Z: Bardock - The Father of Goku (D...


Then make the personalized ranking API call.

In [51]:
# Convert user to string:
user_id = str(rerank_user)
rerank_item_list = []
for item in rerank_items:
    rerank_item_list.append(str(item))
    
# Get recommended reranking
get_recommendations_response_rerank = personalize_runtime.get_personalized_ranking(
        campaignArn = rerank_campaign_arn,
        userId = user_id,
        inputList = rerank_item_list
)

Now add the reranked items as a second column to the original dataframe, for a side-by-side comparison.

In [52]:
ranked_list = []
item_list = get_recommendations_response_rerank['personalizedRanking']
for item in item_list:
    movie = get_movie_by_id(item['itemId'])
    ranked_list.append(movie)
ranked_df = pd.DataFrame(ranked_list, columns = ['Re-Ranked'])
rerank_df = pd.concat([rerank_df, ranked_df], axis=1)
rerank_df

,Un-Ranked,Re-Ranked
0,Babylon 5: The Legend of the Rangers: To Live ...,"First Wives Club, The (1996)"
1,"First Wives Club, The (1996)",Midnight in the Garden of Good and Evil (1997)
2,Boniface's Holiday (1965),Metropolitan (1990)
3,Exotica (1994),Exotica (1994)
4,Horrors of Spider Island (Ein Toter Hing im Ne...,"Exterminating Angel, The (Ãngel exterminador,..."
5,The Emperor's Club (2002),Incredibly True Adventure of Two Girls in Love...
6,"Ladykillers, The (2004)",Boniface's Holiday (1965)
7,"Fall of the House of Usher, The (House of Ushe...",Some Folks Call It a Sling Blade (1993)
8,"Queen of Versailles, The (2012)",Cocoon: The Return (1988)
9,Dragon Ball Z: Bardock - The Father of Goku (D...,She Wore a Yellow Ribbon (1949)


You can see above how each entry was re-ordered based on the model's understanding of the user. This is a popular task when you have a collection of items to surface a user, a list of promotions for example.

## Batch recommendations <a class="anchor" id="batch"></a>
[Back to top](#top)

There are many cases where you may want to have a larger dataset of exported recommendations. Recently, Amazon Personalize launched batch recommendations as a way to export a collection of recommendations to S3. In this example, we will walk through how to do this for the HRNN solution. For more information about batch recommendations, please see the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/getting-recommendations.html#recommendations-batch). This feature applies to all recipes, but the output format will vary.

A simple implementation looks like this:

```python
import boto3

personalize_rec = boto3.client(service_name='personalize')

personalize_rec.create_batch_inference_job (
    solutionVersionArn = "Solution version ARN",
    jobName = "Batch job name",
    roleArn = "IAM role ARN",
    jobInput = 
       {"s3DataSource": {"path": S3 input path}},
    jobOutput = 
       {"s3DataDestination": {"path":S3 output path"}}
)
```

The SDK import, the solution version arn, and role arns have all been determined. This just leaves an input, an output, and a job name to be defined.

Starting with the input for HRNN, it looks like:


```JSON
{"userId": "4638"}
{"userId": "663"}
{"userId": "3384"}
```

This should yield an output that looks like this:

```JSON
{"input":{"userId":"4638"}, "output": {"recommendedItems": ["296", "1", "260", "318"]}}
{"input":{"userId":"663"}, "output": {"recommendedItems": ["1393", "3793", "2701", "3826"]}}
{"input":{"userId":"3384"}, "output": {"recommendedItems": ["8368", "5989", "40815", "48780"]}}
```

The output is a JSON Lines file. It consists of individual JSON objects, one per line. So we will need to put in more work later to digest the results in this format.

### Building the input file

When you are using the batch feature, you specify the users that you'd like to receive recommendations for when the job has completed. The cell below will again select a few random users and will then build the file and save it to disk. From there, you will upload it to S3 to use in the API call later.

In [37]:
# We will use the same users from before
users
# Write the file to disk
json_input_filename = "json_input.json"
with open(data_dir + "/" + json_input_filename, 'w') as json_input:
    for user_id in users:
        json_input.write('{"userId": "' + str(user_id) + '"}\n')

In [38]:
# Showcase the input file:
!cat $data_dir"/"$json_input_filename

{"userId": "381"}
{"userId": "573"}
{"userId": "431"}


Upload the file to S3 and save the path as a variable for later.

In [39]:
# Upload files to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(json_input_filename).upload_file(data_dir+"/"+json_input_filename)
s3_input_path = "s3://" + bucket_name + "/" + json_input_filename
print(s3_input_path)

s3://832194813872-us-east-1-personalizepocvod/json_input.json


Batch recommendations read the input from the file we've uploaded to S3. Similarly, batch recommendations will save the output to file in S3. So we define the output path where the results should be saved.

In [40]:
# Define the output path
s3_output_path = "s3://" + bucket_name + "/"
print(s3_output_path)

s3://832194813872-us-east-1-personalizepocvod/


Now just make the call to kick off the batch export process.

In [41]:
batchInferenceJobArn = personalize.create_batch_inference_job (
    solutionVersionArn = userpersonalization_solution_version_arn,
    jobName = "VOD-POC-Batch-Inference-Job-UserPersonalization_" + str(round(time.time()*1000)),
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

Run the while loop below to track the status of the batch recommendation call. This can take around 30 minutes to complete, because Personalize needs to stand up the infrastructure to perform the task. We are testing the feature with a dataset of only 3 users, which is not an efficient use of this mechanism. Normally, you would only use this feature for bulk processing, in which case the efficiencies will become clear.

In [42]:
current_time = datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

Import Started on:  12:28:45 AM
DatasetInferenceJob: CREATE PENDING
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInfer

In [43]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket_name, export_name, data_dir+"/"+export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open(data_dir+"/"+export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the artists
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            movie = get_movie_by_id(item)
            recommendation_list.append(movie)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

,User: 381,User: 573,User: 431
0,"Pursuit of Happyness, The (2006)",Up (2009),Phenomenon (1996)
1,Into the Wild (2007),"Dark Knight, The (2008)",Sliding Doors (1998)
2,Slumdog Millionaire (2008),District 9 (2009),"Crying Game, The (1992)"
3,Scarface (1983),Raiders of the Lost Ark (Indiana Jones and the...,"Time to Kill, A (1996)"
4,Walk the Line (2005),"Godfather: Part II, The (1974)",Sense and Sensibility (1995)
5,"Notebook, The (2004)",Inglourious Basterds (2009),Leaving Las Vegas (1995)
6,Atonement (2007),"Pan's Labyrinth (Laberinto del fauno, El) (2006)",Conspiracy Theory (1997)
7,Harry Potter and the Deathly Hallows: Part 1 (...,WALLÂ·E (2008),Jerry Maguire (1996)
8,X-Men (2000),"Lord of the Rings: The Return of the King, The...","Postman, The (Postino, Il) (1994)"
9,Top Gun (1986),"Godfather, The (1972)","Bodyguard, The (1992)"


## Wrap up <a class="anchor" id="wrapup"></a>
[Back to top](#top)

With that you now have a fully working collection of models to tackle various recommendation and personalization scenarios, as well as the skills to manipulate customer data to better integrate with the service, and a knowledge of how to do all this over APIs and by leveraging open source data science tools.

Use these notebooks as a guide to getting started with your customers for POCs. As you find missing components, or discover new approaches, cut a pull request and provide any additional helpful components that may be missing from this collection.

You'll want to make sure that you clean up all of the resources deployed during this POC. We have provided a separate notebook which shows you how to identify and delete the resources in `06_Clean_Up_Resources.ipynb`.

In [53]:
%store event_tracker_arn
%store TRACKING_ID
%store batchInferenceJobArn
%store items_df

Stored 'event_tracker_arn' (str)
Stored 'TRACKING_ID' (str)
Stored 'items_df' (DataFrame)
